In [2]:
import requests
import pandas as pd
import os
import json


In [20]:

# URL for Wikipedia list of S&P 500 companies
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Read the data into a DataFrame
table = pd.read_html(url, header=0)[0]
sp_500_tickers_list = table['Symbol'].tolist()  # List of tickers from Wikipedia
print(sp_500_tickers_list)
print(sp_500_tickers_list[:10])
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [ ]:
import requests
from tqdm import tqdm

# get S&P 500 companies
url = "https://seeking-alpha.p.rapidapi.com/analysis/v2/list"

total_len = 0
valid = 0

# sample 100 companies
for ticker in tqdm(sp_500_tickers_list[:100]):
    querystring = {
        "id":ticker, 
        "since":"1672531200", # 2023-01-01
        "until": "1675209540", # 2023-02-01
        "size": "40"
        }
# 'x-rapidapi-key': "13f98f0478msha8ec97b6a805b1fp174175jsn98c8458d4397"
    headers = {
        "x-rapidapi-key": "ad0687c4eamshf7fa27f274fdb41p1e7bd0jsncb21f54850f1",
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }
    try:
        response = requests.get(url, headers=headers, params=querystring).json()
    except:
        continue

    total_len += len(response['data'])
    valid += 1

100%|██████████| 100/100 [04:05<00:00,  2.45s/it]


In [52]:
total_len

356

In [44]:
response['data'][-1]

{'id': '4568298',
 'type': 'article',
 'attributes': {'publishOn': '2023-01-06T11:15:18-05:00',
  'isLockedPro': False,
  'commentCount': 7,
  'gettyImageUrl': 'https://static.seekingalpha.com/cdn/s3/uploads/getty_images/514543268/image_514543268.jpg',
  'videoPreviewUrl': None,
  'videoDuration': None,
  'themes': {},
  'title': 'Constellation Brands: Demand And Margin Concerns Drive A Selloff',
  'structuredInsights': None,
  'isPaywalled': False},
 'relationships': {'author': {'data': {'id': '24361', 'type': 'author'}},
  'sentiments': {'data': [{'id': '429825', 'type': 'sentiment'}]},
  'primaryTickers': {'data': [{'id': '2497', 'type': 'tag'}]},
  'secondaryTickers': {'data': []},
  'otherTags': {'data': []}},
 'links': {'self': '/article/4568298-constellation-brands-stock-demand-margin-concerns-drive-selloff'}}

In [56]:
import requests


url = "https://seeking-alpha.p.rapidapi.com/market/get-realtime-prices"

querystring = {"symbols":"aapl"}

# 'x-rapidapi-key': "13f98f0478msha8ec97b6a805b1fp174175jsn98c8458d4397"
headers = {
        "x-rapidapi-key": "13f98f0478msha8ec97b6a805b1fp174175jsn98c8458d4397",
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

response = requests.get(url, headers=headers, params=querystring)

print(response)

<Response [204]>


In [57]:
import json

# Save data to a JSON file
with open('get-realtime-prices.json', 'w') as json_file:
    json.dump(response.json(), json_file)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
path = './sp500_total_constituents.csv'

try:
    df = pd.read_csv(path, encoding = 'utf-8')
    cik = df['CIK'].drop_duplicates().tolist()
    ticker = df['Symbol'].tolist()
    cik_ticker = dict(zip(cik, ticker))
except UnicodeDecodeError:
    df = pd.read_csv(path, encoding = 'ISO-8859-1')
    cik = df['CIK'].drop_duplicates().tolist()
    ticker = df['Symbol'].tolist()
    cik_ticker = dict(zip(cik, ticker))
print(cik_ticker)

{804328: 'QCOM', 49826: 'ITW', 715957: 'D', 1018963: 'ATI', 46080: 'HAS', 107263: 'WMB', 769397: 'ADSK', 32604: 'EMR', 795266: 'KBH', 909832: 'COST', 1568100: 'PD', 12927: 'BA', 23217: 'CAG', 87347: 'SLB', 101778: 'MRO', 1817358: 'ASO', 8818: 'AVY', 92380: 'LUV', 91440: 'SNA', 723125: 'MU', 1674862: 'ASH', 51253: 'IFF', 97476: 'TXN', 40545: 'GE', 38777: 'BEN', 318154: 'AMGN', 815097: 'CCL', 885639: 'KSS', 1012100: 'SEE', 1284812: 'CNS', 814453: 'NWL', 85961: 'R', 62709: 'MMC', 1881551: 'STI', 874766: 'HIG', 764622: 'PNW', 60667: 'LOW', 319201: 'KLAC', 37996: 'F', 823768: 'WM', 60519: 'LPX', 773840: 'HON', 827052: 'EIX', 1800: 'ABT', 876437: 'MTG', 70318: 'THC', 106640: 'WHR', 1164727: 'NEM', 827099: 'CPWR', 40987: 'GPC', 4962: 'AXP', 59558: 'LNC', 702165: 'NSC', 899051: 'ALL', 29905: 'DOV', 56873: 'KR', 49196: 'HBAN', 1666700: 'DD', 27419: 'TGT', 1031296: 'FE', 922224: 'PPL', 63276: 'MAT', 34088: 'XOM', 79879: 'PPG', 109198: 'TJX', 1390777: 'BK', 40704: 'GIS', 831001: 'C', 1474098: 'PE

Get Historical Price Data


In [4]:
import requests
import os
from tqdm import tqdm

total_len = 0
valid = 0
save_folder = "SP500-historical-prices_2012-2024"
for ticker in tqdm(cik_ticker.values()):
	file_name = f'{ticker}.json'
	file_path = os.path.join(save_folder, file_name)
	if os.path.exists(file_path):
		print(f"Skipping {ticker}: {file_path} already exists.")
		continue
    
	url = "https://seeking-alpha.p.rapidapi.com/symbols/get-historical-prices"
	querystring = {"symbol":ticker,"show_by":"day","start":"2012-01-01","end":"2024-09-01","sort":"as_of_date"}

	headers = {
		"x-rapidapi-key": "13f98f0478msha8ec97b6a805b1fp174175jsn98c8458d4397",
		"x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
	}

	try:
		response = requests.get(url, headers=headers, params=querystring).json()
		# response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
    
		# if 'application/json' in response.headers.get('Content-Type', ''):
		# 	data = response.json()  # Parse JSON if content is JSON
		# 	print(data)
		# else:
		# 	print("Response is not JSON")
		# 	print(response.text)  # Debug raw response
	except:
		continue
	total_len += len(response['data'])
	valid += 1

	if not os.path.exists(save_folder):
		os.makedirs(save_folder)
	with open(f'SP500-historical-prices_2012-2024/{ticker}.json', 'w') as json_file:
		json.dump(response, json_file)

print("The number of firm retrieved:", total_len)


  0%|          | 0/549 [00:00<?, ?it/s]

Skipping QCOM: SP500-historical-prices_2012-2024/QCOM.json already exists.
Skipping ITW: SP500-historical-prices_2012-2024/ITW.json already exists.
Skipping D: SP500-historical-prices_2012-2024/D.json already exists.
Skipping ATI: SP500-historical-prices_2012-2024/ATI.json already exists.
Skipping HAS: SP500-historical-prices_2012-2024/HAS.json already exists.
Skipping WMB: SP500-historical-prices_2012-2024/WMB.json already exists.
Skipping ADSK: SP500-historical-prices_2012-2024/ADSK.json already exists.
Skipping EMR: SP500-historical-prices_2012-2024/EMR.json already exists.
Skipping KBH: SP500-historical-prices_2012-2024/KBH.json already exists.
Skipping COST: SP500-historical-prices_2012-2024/COST.json already exists.
Skipping PD: SP500-historical-prices_2012-2024/PD.json already exists.
Skipping BA: SP500-historical-prices_2012-2024/BA.json already exists.
Skipping CAG: SP500-historical-prices_2012-2024/CAG.json already exists.
Skipping SLB: SP500-historical-prices_2012-2024/SLB.j

100%|██████████| 549/549 [25:32<00:00,  2.79s/it]

The number of firm retrieved: 1455301


Get Earning Call Transcript Calender Data

In [28]:
import requests

from tqdm import tqdm

total_len = 0
valid = 0
save_folder = "SP500-earning-call-calender_2006-2013"
for ticker in tqdm(cik_ticker.values()):
    file_name = f'{ticker}.json'
    file_path = os.path.join(save_folder, file_name)
    if os.path.exists(file_path):
        print(f"Skipping {ticker}: {file_path} already exists.")
        continue
    
    url = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/list"

    querystring = {"id":f"{ticker}"
                ,"until":"1388534399" #2013-12-31
                ,"since":"1136073600" #2006-01-01
                ,"size":"40"
                ,"number":"1"}

    headers = {
		"x-rapidapi-key": "13f98f0478msha8ec97b6a805b1fp174175jsn98c8458d4397",
		"x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
	}
    try:
        response = requests.get(url, headers=headers, params=querystring).json()
        # response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
    
        # if 'application/json' in response.headers.get('Content-Type', ''):
        #     data = response.json()  # Parse JSON if content is JSON
        #     print(data)
        # else:
        #     print("Response is not JSON")
        #     print(response.text)  # Debug raw response
    except:
        continue

    total_len += len(response['data'])
    valid += 1


    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    with open(f'SP500-earning-call-calender_2006-2013/{ticker}.json', 'w') as json_file:
        json.dump(response, json_file)

print("The number of firm retrieved:", total_len)


  0%|          | 0/549 [00:00<?, ?it/s]

100%|██████████| 549/549 [17:29<00:00,  1.91s/it]

The number of firm retrieved: 13410


In [29]:
total_len = 0
valid = 0
save_folder = "SP500-earning-call-calender_2014-2024"

for ticker in tqdm(cik_ticker.values()):
    file_name = f'{ticker}.json'
    file_path = os.path.join(save_folder, file_name)
    if os.path.exists(file_path):
        print(f"Skipping {ticker}: {file_path} already exists.")
        continue
    
    url = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/list"

    querystring = {"id":f"{ticker}"
                ,"until":"1725148800"  #2024-09-01
                ,"since":"1388534400" #2014-01-01
                ,"size":"40"
                ,"number":"1"}

    headers = {
		"x-rapidapi-key": "13f98f0478msha8ec97b6a805b1fp174175jsn98c8458d4397",
		"x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
	}
    try: 
        response = requests.get(url, headers=headers, params=querystring).json()
    except:
        continue
    total_len += len(response['data'])
    valid += 1


    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    with open(f'SP500-earning-call-calender_2014-2024/{ticker}.json', 'w') as json_file:
        json.dump(response, json_file)

print("The number of firm retrieved:", total_len)

100%|██████████| 549/549 [20:19<00:00,  2.22s/it]  

The number of firm retrieved: 21157


Merge Json

In [30]:
import os
import json

# Paths to the two folders. Merge order is important. 
folder1 = "/Users/apple/PROJECT/Code_4_10k/SP500-earning-call-calender_2014-2024" 
folder2 = "/Users/apple/PROJECT/Code_4_10k/SP500-earning-call-calender_2006-2013"
output_folder = "/Users/apple/PROJECT/Code_4_10k/SP500-earning-call-calender_2006-2024"

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get a list of files in both folders
files1 = set(os.listdir(folder1))
files2 = set(os.listdir(folder2))

# Find files with matching names
common_files = files1.intersection(files2)

# Merge files with the same names
for file_name in common_files:
    file1_path = os.path.join(folder1, file_name)

    

    file2_path = os.path.join(folder2, file_name)
    output_file_path = os.path.join(output_folder, file_name)
    
    try:
        # Read and parse the JSON files
        with open(file1_path, 'r') as f1, open(file2_path, 'r') as f2:
            data1 = json.load(f1)
            data2 = json.load(f2)
        # print('data1', data1)
        # print('data2', data2)
        merged_data = data1['data'] + data2['data'] 

        # Write the merged data to the output folder
        with open(output_file_path, 'w') as output_file:
            json.dump(merged_data, output_file, indent=4)
        
        print(f"Merged {file_name} successfully.")
    except Exception as e:
        print(f"Error processing {file_name}: {e}")

print("Merging completed.")


Merged AMZN.json successfully.
Merged DUK.json successfully.
Merged ICE.json successfully.
Merged DHR.json successfully.
Merged HUM.json successfully.
Merged JCI.json successfully.
Merged PCAR.json successfully.
Merged ANET.json successfully.
Merged WBD.json successfully.
Merged MTCH.json successfully.
Merged EMR.json successfully.
Merged SYF.json successfully.
Merged LNC.json successfully.
Merged WMB.json successfully.
Merged HAS.json successfully.
Merged QRVO.json successfully.
Merged HSIC.json successfully.
Merged PTC.json successfully.
Merged DLTR.json successfully.
Merged BR.json successfully.
Merged TEX.json successfully.
Merged EQIX.json successfully.
Merged FIS.json successfully.
Merged PNC.json successfully.
Merged BMY.json successfully.
Merged DVA.json successfully.
Merged MO.json successfully.
Merged LNT.json successfully.
Merged BAC.json successfully.
Merged INVH.json successfully.
Merged VNT.json successfully.
Merged UHS.json successfully.
Merged ROST.json successfully.
Me

In [12]:
with open('get-trasnacripts-appl-2006-2013.json', 'r') as f1, open('get-trasnacripts-appl-2014-2024.json', 'r') as f2:
    json1 = json.load(f2)
    json2 = json.load(f1)
print(json1)

# Merge the "data" arrays
merged_data = json1["data"] + json2["data"]

# Save the merged JSON to a new file
with open('merged.json', 'w') as outfile:
    json.dump(merged_data, outfile, indent=4)

{'data': [{'id': '4688870', 'type': 'transcript', 'attributes': {'publishOn': '2024-05-02T20:30:10-04:00', 'isLockedPro': False, 'commentCount': 1, 'gettyImageUrl': None, 'videoPreviewUrl': None, 'videoDuration': None, 'themes': {}, 'title': 'Apple Inc. (AAPL) Q2 2024 Earnings Call Transcript', 'structuredInsights': None, 'isPaywalled': False}, 'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}}, 'sentiments': {'data': []}, 'primaryTickers': {'data': [{'id': '146', 'type': 'tag'}]}, 'secondaryTickers': {'data': []}, 'otherTags': {'data': [{'id': '49', 'type': 'tag'}]}}, 'links': {'self': '/article/4688870-apple-inc-aapl-q2-2024-earnings-call-transcript'}}, {'id': '4666956', 'type': 'transcript', 'attributes': {'publishOn': '2024-02-01T20:17:02-05:00', 'isLockedPro': False, 'commentCount': 18, 'gettyImageUrl': None, 'videoPreviewUrl': None, 'videoDuration': None, 'themes': {}, 'title': 'Apple Inc. (AAPL) Q1 2024 Earnings Call Transcript', 'structuredInsights': None, 